In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.signal import butter, filtfilt
import os
import imageio
from tqdm import tqdm

# Notebook for the SPECT preprocessing steps
## 1. Gaussian isotropic filter

In [ ]:
def apply_gaussian_filter(input_file, output_dir):
    fwhm_mm = 18
    # Load the NIfTI file
    img = nib.load(input_file)
    data = img.get_fdata()

    # Get voxel dimensions
    voxel_sizes = img.header.get_zooms()
    #print("voxel_sizes: ", voxel_sizes)

    # Convert FWHM from mm to voxel units
    fwhm_voxel = fwhm_mm / np.array(voxel_sizes)

    # Apply Gaussian filter
    filtered_data = gaussian_filter(data, sigma=fwhm_voxel / (2*np.sqrt(2*np.log(2))))

    # Save the filtered data to a new NIfTI file
    filtered_img = nib.Nifti1Image(filtered_data, img.affine, img.header)
    # check if the output dir exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    sufix = "_gaussian.nii.gz"
    original_name = input_file.split("/")[-1]
    current_original_name = original_name.split(".")[-3]
    prefixed_name = f"{current_original_name}{sufix}"
    output_file = output_dir + "/" + prefixed_name
    print("output_file: ", output_file)
    nib.save(filtered_img, output_file)
    
    return output_file

# # Example usage
# input_file = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/test/control/spect/3104/preprocessed/PPMI_3104_NM_Reconstructed_DaTSCAN_Br_20121011134355542_1_S117556_spect_resampled.nii.gz"
# output_file = '/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/test/control/spect/3104/preprocessed2/filtered_output_gaussian.nii'
# apply_gaussian_filter(input_file, output_file, fwhm_mm)

In [ ]:
def apply_gaussian_filter_pngs(image_path, output_path, sigma):
    # Read the image
    image = imageio.imread(image_path)
    
    # Apply the Gaussian filter
    filtered_image = gaussian_filter(image, sigma=sigma)
    
    # Save the filtered image
    imageio.imwrite(output_path, filtered_image)

## 2. Reconstruction algorithms
a) Butterworth filter of 5th order with a cutoff frequency of 0.6 cycles/pixel 

In [ ]:
def butterworth_filter(input_file, output_dir, order=5, cutoff=0.1):
    # Load the NIfTI file
    img = nib.load(input_file)
    data = img.get_fdata()

    # Get voxel dimensions
    voxel_sizes = img.header.get_zooms()[:3]  # Extract voxel dimensions
    
    # Calculate sampling rate based on voxel dimensions
    sampling_rate = 1 / min(voxel_sizes)

    # Calculate Nyquist frequency
    nyquist_freq = 0.5 * sampling_rate  # Nyquist frequency

    #print("Nyquist Frequency:", nyquist_freq)  # Debugging statement

    # Normalize cutoff frequency
    normalized_cutoff = cutoff / nyquist_freq

    #print("Normalized Cutoff Frequency:", normalized_cutoff)  # Debugging statement

    # Check if normalized cutoff is within valid range
    if normalized_cutoff >= 1:
        raise ValueError("Cutoff frequency exceeds Nyquist frequency.")

    # Design Butterworth filter
    b, a = butter(order, normalized_cutoff, btype='low', analog=False)

    # Apply filter to each slice of the data
    filtered_data = np.zeros_like(data)
    for i in range(data.shape[-1]):
        filtered_data[..., i] = filtfilt(b, a, data[..., i], axis=0)
        
    # check if the output dir exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    sufix = "_butterworth.nii.gz"
    original_name = input_file.split("/")[-1]
    current_original_name = original_name.split(".")[-3]
    prefixed_name = f"{current_original_name}{sufix}"
    output_file = output_dir + "/" + prefixed_name
    # Save the filtered data back to a nii file
    filtered_nii = nib.Nifti1Image(filtered_data, affine=img.affine)
    nib.save(filtered_nii, output_file)

    return output_file

a.1) Uniform post-reconstruction attenuation correction according to Chang (mu=0.12/cm), no scatter correction

In [ ]:
def apply_attenuation_correction(input_file, output_dir, attenuation_coefficient=0.12):
    # Load the SPECT NIfTI file
    img = nib.load(input_file)
    # Extract image data from the NIfTI file
    spect_data = img.get_fdata()
    
    
    # Apply attenuation correction using Chang's method
    corrected_data = spect_data * np.exp(-attenuation_coefficient)
    
    # check if the output dir exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    sufix = "_attenuated.nii.gz"
    original_name = input_file.split("/")[-1]
    current_original_name = original_name.split(".")[-3]
    prefixed_name = f"{current_original_name}{sufix}"
    output_file = output_dir + "/" + prefixed_name
    # Save the filtered data back to a nii file
    filtered_nii = nib.Nifti1Image(corrected_data, affine=img.affine)
    nib.save(filtered_nii, output_file)
    
    return output_file

**Main**
* For Parkinson and Control groups 

In [ ]:
#getting all the related cases
root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/FinalExtension/data/classifier"
split = "train"
group = "parkinson"
modality = "spect"
current_path = root_path + "/" + group + "_" + split + "/" + modality + "_png"
cases = sorted(os.listdir(current_path))
print(len(cases))

* For SWEDD group

Temporal

In [ ]:
current_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/swedd/spect/swedd_only_spect_v2/PPMI"
group = "swedd"
modality = "spect"
cases = sorted(os.listdir(current_path))
print(len(cases))

Hasta aca

In [ ]:
#getting all the related cases
root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered"
group = "swedd"
modality = "spect"
current_path = root_path + "/" + group + "/" + modality
cases = sorted(os.listdir(current_path))
print(len(cases))

In [ ]:
for case in cases:
    print(case)
    preprocessed_case_path = current_path + "/" + case + "/" + "preprocessed"
    output_dir = current_path + "/" + case + "/" + "preprocessed2"
    files = os.listdir(preprocessed_case_path)
    nii_file = [file for file in files if file.endswith("_resampled.nii.gz")][0] 
    #step 1 to preprocess (apply gaussian filter)
    nii_file_path = preprocessed_case_path + "/" + nii_file
    input_file = nii_file_path
    output_file = apply_gaussian_filter(input_file, output_dir)
    #step 2 to preprocess (apply Butterworth reconstruction filter)
    output_file = butterworth_filter(input_file=output_file, output_dir=output_dir, order=5, cutoff=0.1)
    #step 3 to preprocess (attenuation correction)
    corrected_data = apply_attenuation_correction(output_file, output_dir, attenuation_coefficient=0.12) # Assuming linear attenuation coefficient of 0.12/cm
    
print("done!")
    

### Making the preprocessing over single cases

In [ ]:
current_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/train/parkinson/spect/3863/"
files = os.listdir(current_path)
nii_file = [file for file in files if file.endswith("_resampled.nii.gz")][0] 
#step 1 to preprocess (apply gaussian filter)
nii_file_path = preprocessed_case_path + "/" + nii_file
input_file = nii_file_path
output_dir = current_path + "preprocessed2"
output_file = apply_gaussian_filter(input_file, output_dir)
#step 2 to preprocess (apply Butterworth reconstruction filter)
output_file = butterworth_filter(input_file=output_file, output_dir=output_dir, order=5, cutoff=0.1)
#step 3 to preprocess (attenuation correction)
corrected_data = apply_attenuation_correction(output_file, output_dir, attenuation_coefficient=0.12)

### Making the preprocessing over PNG's files

In [ ]:
# Example usage
# Define the FWHM and calculate sigma
FWHM = 18  # in mm
sigma = FWHM / 2.3548
root_path = "/home/Data/franklin/Doctorado/parkinson/projects/T1-SPECT-PD-translation/imgs_results/t12spect/"
split = "train"
group = "control"
cases = sorted(os.listdir(root_path + split + "/" + group + "/"))
print("total of cases: ", len(cases))

for case in tqdm(cases):
    print("case: ", case)
    case_path = root_path + split + "/" + group + "/" + case + "/"
    images = sorted(os.listdir(case_path))
    for image in images:
        general_inf = image.split(".")[0]
        img_num = int(general_inf.split("_")[-1])
        if img_num >= 57 and img_num <= 66:
            input_image_path = case_path + image
            directory = root_path + "preprocessed/" + split + "/" + group + "/" + case
            
            if not os.path.exists(directory):
                os.makedirs(directory)
            else:
                None
                
            output_image_path = directory + "/" + image
            # print("input_image_path: ", input_image_path)
            # print("output_image_path: ", output_image_path)
            apply_gaussian_filter_pngs(input_image_path, output_image_path, sigma)
        else:
            None